<a href="https://colab.research.google.com/github/MevrouwHelderder/Assignments/blob/main/Assignment_Shark_Attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [132]:
import re

In [20]:
# Importing the essentials
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
# importing the dataframe
path = '/content/drive/MyDrive/attacks.csv'
attacks = pd.read_csv(path, encoding="ISO-8859-1")
# Dropping columns.
attacks_drop_columns = attacks.drop(columns=['Date', 'Year','Country', 'Area', 'Location','Name', 'Sex ', 'Time', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'])
# Renaming columns.
attacks_renamed = attacks_drop_columns.rename(columns={'Case Number': 'Case', 'Fatal (Y/N)': 'Fatal', 'Species ': 'Species' })
# Dropping rows.
# Making a copy to prevent view vs copy issues later on.
relevant_columns = list(attacks_renamed.columns[1:])
attacks_drop_rows = attacks_renamed.dropna(subset=relevant_columns, how='all').copy()
# Preparing the functions for locating and adjusting the missing values.
def print_separator(sep, num, msg):
  print("\n")
  print(sep * num)
  print(f"{msg}")
  print(sep * num)
# TACTIC A: find unique values
def look_at_unique_values(column): 
  unique_values_cutoff = 50
  unique_values = column.unique()
  num_unique_values = len(unique_values)
  if num_unique_values == len(column):
    print(f"Each value in the column is unique (total: {num_unique_values})")
  elif num_unique_values < unique_values_cutoff: 
    print(f"Less than {unique_values_cutoff} unique values:")
    try:
      sorted = np.sort(unique_values)
      print("Values are sorted")
      display(list(sorted))
    except:
      print("Could not sort values")
      display(list(unique_values))
  else:
    print(f"More than {unique_values_cutoff} unique values (total: {num_unique_values})")
# TACTIC B: look at the edges
def look_at_edges(df, column_name):
  # inner function
  def show_head_and_tail(values):
      num_items_to_slice = 10
      display(list(values)[:num_items_to_slice]) 
      display(list(values)[-num_items_to_slice:]) 
  column = df[column_name]
  unique_values = column.unique()
  try: 
      sorted = np.sort(unique_values)
      print("Unique values sorted, head and tail:")
      show_head_and_tail(sorted)
  except TypeError as error:
      print(f"Could not sort values: {error}")
      print("..so let's try filtering NULL values and then sorting")
      print("..there could be a black sheep in the null values")
      non_null_uniques = df.loc[~df[column_name].isnull(), column_name].unique()
      sorted = np.sort(non_null_uniques)
      show_head_and_tail(sorted)
# TACTIC C: casting to a type to see if all the values match the needed type
def cast_to_type(column, maybe_type):
  try:
    column.astype(maybe_type)
    print(f"Casting to {maybe_type} was successful")
  except ValueError as error:
    print(f"Could not cast to {maybe_type}: {error}")
# TACTIC D: display the value count of the column
def value_count(column):
  display(column.value_counts(dropna=False))
# FUNCTION TO CHECK THE DATAFRAME FOR ALL FOUR TACTICS
def find_non_default_missing_values(df, column_name, maybe_type):
  long_separator_amount = 80
  short_separator_amount = 40
  # Print the header
  print_separator("*", long_separator_amount, f"Finding non default missing values for column \"{column_name}\"")
  print(f"Column \"{column_name}\" has datatype: {df.dtypes[column_name]}")
  column = df[column_name]  
  # A
  print_separator("-", short_separator_amount, "A: Looking at unique values")
  look_at_unique_values(column)
  # B
  print_separator("-", short_separator_amount, "B: Sorting and looking at the edges")
  look_at_edges(df, column_name)
  # C
  print_separator("-", short_separator_amount, f"C: Casting to type: {maybe_type}")
  cast_to_type(column, maybe_type)
  # D
  print_separator("-", short_separator_amount, "D: Looking at frequency\nAll default-NULL values will be bunched together as NaN")
  value_count(column)
  print("\n")
# Function to replace non-default NULL values with default NULL values.
# ⚠️ Mutates df
def replace_value(df, column_name, missing_old, missing_new):
  df[column_name] = df[column_name].replace({missing_old: missing_new})
# Function to display the default NULL values in the column.
def display_default_null_values(df, column_name):
  nulls = df.loc[df[column_name].isnull()]
  print(f"Number of default null values in \"{column_name}\": {len(nulls)}")
# Easier to type
nat = np.datetime64('nat')
# Making a deep copy of the dataframe.
attacks_clean = attacks_drop_rows.copy(deep=True)
# Lowercase all strings and strip whitespace and/or quotationmarks around strings
attacks_clean = attacks_clean.applymap(lambda x: x.lower() if isinstance(x, str) else x)
attacks_clean = attacks_clean.applymap(lambda x: x.strip('" ') if type(x)==str else x)
# Column Type
attacks_clean = attacks_clean.replace({'Type' : {np.nan : None, 'Boating' : 'Boat', 'Boatomg' : 'Boat'}})
# Column Activity
attacks_clean['Activity'] = attacks_clean['Activity'].replace({'.': None})

In [187]:
# NOTE!!!! CHANGE THIS LATER, CHANGE THE NAMES OF THE DF AND COLUMNS IN THE FUNCTIONS TO FOLLOW!!!!!
# Making a copy for now, to not have to do the previous steps over and over again when fiddling around.

attacks_temp = attacks_clean.copy(deep=True)

In [142]:
attacks_clean

,Case,Type,Activity,Age,Injury,Fatal,Species
0,2018.06.25,boating,paddling,57,"no injury to occupant, outrigger canoe and pad...",n,white shark
1,2018.06.18,unprovoked,standing,11,minor injury to left thigh,n,NaN
2,2018.06.09,invalid,surfing,48,injury to left lower leg from surfboard skeg,n,NaN
3,2018.06.08,unprovoked,surfing,NaN,minor injury to lower leg,n,2 m shark
4,2018.06.04,provoked,free diving,NaN,lacerations to leg & hand shark provoked incident,n,"tiger shark, 3m"
...,...,...,...,...,...,...,...
6297,nd.0005,unprovoked,diving,NaN,fatal,y,NaN
6298,nd.0004,unprovoked,pearl diving,NaN,fatal,y,NaN
6299,nd.0003,unprovoked,swimming,NaN,fatal,y,NaN
6300,nd.0002,unprovoked,NaN,NaN,fatal,y,NaN


In [192]:
attacks_temp

,Case,Type,Activity,Age,Injury,Fatal,Species
0,2018.06.25,boating,paddling,57,"no injury to occupant, outrigger canoe and pad...",n,white shark
1,2018.06.18,unprovoked,standing,11,minor injury to left thigh,n,no species provided
2,2018.06.09,invalid,surfing,48,injury to left lower leg from surfboard skeg,n,no species provided
3,2018.06.08,unprovoked,surfing,NaN,minor injury to lower leg,n,m shark
4,2018.06.04,provoked,free diving,NaN,lacerations to leg & hand shark provoked incident,n,tiger shark
...,...,...,...,...,...,...,...
6297,nd.0005,unprovoked,diving,NaN,fatal,y,no species provided
6298,nd.0004,unprovoked,pearl diving,NaN,fatal,y,no species provided
6299,nd.0003,unprovoked,swimming,NaN,fatal,y,no species provided
6300,nd.0002,unprovoked,NaN,NaN,fatal,y,no species provided


In [189]:
# Column Species
# Replace the nulls 
attacks_temp['Species'] = attacks_temp['Species'].fillna("no species provided")

In [165]:
# Function to replace values in the column
# attacks_temp['Species'] = attacks_temp['Species'].apply(lambda x: ("shark" in x and re.search("(\S+\s)?shark", x).group()) or ("no species provided" in x and x) or f'check: {x}')


In [190]:
def shark(x): 
  if "shark" in x:
    return re.search("(\S+\s)?shark", x).group()
  elif "no species provided" in x: 
    return x
  else:
    return f'check: {x}'

no_species = "no species provided"

In [186]:
# Try to make it more compact

# attacks_temp['Species'] = attacks_temp['Species'].apply(lambda x: ("shark" in x and shark(x)) or (no_species in x and x) or f'check: {x}')

In [191]:
# Even more compact, with an outside function

attacks_temp['Species'] = attacks_temp['Species'].apply(lambda x: shark(x))

In [ ]:

 # Column fatal
# Check column 'Fatal' for missing values
# find_non_default_missing_values(attacks_clean, 'Fatal', 'bool')
# display(attacks_clean.loc[(attacks_clean['Fatal']== False) | (attacks_clean['Fatal']== True)])
# Replacing values that are clear about their meaning with their boolean equivalents 
# attacks_clean['Fatal'] = attacks_clean['Fatal'].replace(dict.fromkeys(['y'], True))
# attacks_clean['Fatal'] = attacks_clean['Fatal'].replace(dict.fromkeys(['n'], False))
# replace_value(attacks_clean, 'Fatal', 'y', True)
# replace_value(attacks_clean, 'Fatal', 'n', False)
# Change values that are not clear to None for now, might remove them later.
# df[column_name].replace({missing_old: missing_new})
# non_bool = attacks_clean.loc[~((attacks_clean['Fatal'] == False) | (attacks_clean['Fatal'] == True))]
# attacks_clean['Fatal'] = attacks_clean['Fatal'].replace({missing_old: missing_new})
# attacks_clean['Fatal'] = attacks_clean.loc[~((attacks_clean['Fatal'] == False) | (attacks_clean['Fatal'] == True))] = None

In [ ]:
# Column Age
# Check column 'Age' for missing values
# find_non_default_missing_values(attacks_clean, 'Age', 'string')
# Strip the leading/trailing whitespace and quotation marks.
# attacks_clean['Age'] = attacks_clean['Age'].str.strip('"\' \()\\')
# Replace \xa0 and np.nan with None
# attacks_clean['Age'] = attacks_clean['Age'].replace(dict.fromkeys(['\xa0',np.nan], None))
# There are a lot of unclear ages. Since we want to know if there is a difference 
# between children and adults I think it is best to divide them into adult and child
# Transform the column so all clear numericals are in the right categories
# attacks_clean.transform({'Age': str.capitalize, "price": lambda price: round(price * 1.1)})





---



---






#Take [this](https://www.kaggle.com/felipeesc/shark-attack-dataset) dataset. 
Use all the skills you've learned up until now to answer the following questions as well as possible. 

* What are the most dangerous types of sharks to humans? 
* Are children more likely to be attacked by sharks? 
* Are shark attacks where sharks were provoked more or less dangerous? 
* Are certain activities more likely to result in a shark attack? 

If you feel you can't answer a question based on the dataset alone, feel free to find other datasets and use them in answering the questions.

For each answer you give not only answer the question but also write about the assumptions you made in answering the question. If an assumption or decision possibly created a bias please write about this as well.

In [ ]:
# # Importing the essentials
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import re

# %load_ext google.colab.data_table

In [ ]:
# # importing the dataframe
# path = '/content/drive/MyDrive/attacks.csv'

In [ ]:
# attacks = pd.read_csv(path, encoding="ISO-8859-1")

## **Part 1: Rows and columns**

**COLUMNS**: There are columns regarding the date, time and location that seem to be irrelevant to the questions asked.

However, I'm not sure if I might need them later on, for example during imputation.

For now I will remove all seemingly irrelevant columns but I might come back to this later.<br><br>
**ROWS**: There seem to be a lot of rows where all values or all values except the 'Case Number' is NaN.

Since only a case number but no other data is useless in this case let's remove those rows.

In [ ]:
# # Dropping columns.

# attacks_drop_columns = attacks.drop(columns=['Date', 'Year','Country', 'Area', 'Location','Name', 'Sex ', 'Time', 'Investigator or Source', 'pdf', 'href formula', 'href',
#        'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
#        'Unnamed: 23'])

In [ ]:
# # Checking the names of the columns to see if anything needs to be adjusted.

# attacks_drop_columns.columns

In [ ]:
# # Renaming columns.

# attacks_renamed = attacks_drop_columns.rename(columns={'Case Number': 'Case', 'Fatal (Y/N)': 'Fatal', 'Species ': 'Species' })

# # Checking the names of the columns to see if everything went well.

# display (attacks_renamed.columns)

In [ ]:
# # Dropping rows.
# # Making a copy to prevent view vs copy issues later on.

# relevant_columns = list(attacks_renamed.columns[1:])

# attacks_drop_rows = attacks_renamed.dropna(subset=relevant_columns, how='all').copy()


## **Part 2: Missing values**
Time to find default and non-default missing values. 
I will use the tactics and functions we also used in an earlier exercise. 

I realize those functions aren't my own but I figured it would be useless to change a bit and pretend it is my own when it is much more important to show I now how and when to use the functions.
So... credit to Winc and off to the next part!

In [ ]:
# # Preparing the functions for locating and adjusting the missing values.

# def print_separator(sep, num, msg):
#   print("\n")
#   print(sep * num)
#   print(f"{msg}")
#   print(sep * num)

# # TACTIC A: find unique values
# def look_at_unique_values(column): 
#   unique_values_cutoff = 50
#   unique_values = column.unique()
#   num_unique_values = len(unique_values)
#   if num_unique_values == len(column):
#     print(f"Each value in the column is unique (total: {num_unique_values})")
#   elif num_unique_values < unique_values_cutoff: 
#     print(f"Less than {unique_values_cutoff} unique values:")
#     try:
#       sorted = np.sort(unique_values)
#       print("Values are sorted")
#       display(list(sorted))
#     except:
#       print("Could not sort values")
#       display(list(unique_values))
#   else:
#     print(f"More than {unique_values_cutoff} unique values (total: {num_unique_values})")

# # TACTIC B: look at the edges
# def look_at_edges(df, column_name):
#   # inner function
#   def show_head_and_tail(values):
#       num_items_to_slice = 10
#       display(list(values)[:num_items_to_slice]) 
#       display(list(values)[-num_items_to_slice:]) 
#   column = df[column_name]
#   unique_values = column.unique()
#   try: 
#       sorted = np.sort(unique_values)
#       print("Unique values sorted, head and tail:")
#       show_head_and_tail(sorted)
#   except TypeError as error:
#       print(f"Could not sort values: {error}")
#       print("..so let's try filtering NULL values and then sorting")
#       print("..there could be a black sheep in the null values")
#       non_null_uniques = df.loc[~df[column_name].isnull(), column_name].unique()
#       sorted = np.sort(non_null_uniques)
#       show_head_and_tail(sorted)

# # TACTIC C: casting to a type to see if all the values match the needed type
# def cast_to_type(column, maybe_type):
#   try:
#     column.astype(maybe_type)
#     print(f"Casting to {maybe_type} was successful")
#   except ValueError as error:
#     print(f"Could not cast to {maybe_type}: {error}")

# # TACTIC D: display the value count of the column
# def value_count(column):
#   display(column.value_counts(dropna=False))

# # FUNCTION TO CHECK THE DATAFRAME FOR ALL FOUR TACTICS
# def find_non_default_missing_values(df, column_name, maybe_type):
#   long_separator_amount = 80
#   short_separator_amount = 40

#   # Print the header
#   print_separator("*", long_separator_amount, f"Finding non default missing values for column \"{column_name}\"")

#   print(f"Column \"{column_name}\" has datatype: {df.dtypes[column_name]}")

#   column = df[column_name]  

#   # A
#   print_separator("-", short_separator_amount, "A: Looking at unique values")
#   look_at_unique_values(column)

#   # B
#   print_separator("-", short_separator_amount, "B: Sorting and looking at the edges")
#   look_at_edges(df, column_name)

#   # C
#   print_separator("-", short_separator_amount, f"C: Casting to type: {maybe_type}")
#   cast_to_type(column, maybe_type)

#   # D
#   print_separator("-", short_separator_amount, "D: Looking at frequency\nAll default-NULL values will be bunched together as NaN")
#   value_count(column)
#   print("\n")

# # Function to replace non-default NULL values with default NULL values.
# # ⚠️ Mutates df
# def replace_value(df, column_name, missing_old, missing_new):
#   df[column_name] = df[column_name].replace({missing_old: missing_new})

# # Function to display the default NULL values in the column.
# def display_default_null_values(df, column_name):
#   nulls = df.loc[df[column_name].isnull()]
#   print(f"Number of default null values in \"{column_name}\": {len(nulls)}")

# # Easier to type
# nat = np.datetime64('nat')

In [ ]:
# Making a deep copy of the dataframe.
# I understand that it is usually not advised to make a copy but instead work in the original dataframe.
# However, for this assignment I would like to be able to use this later for reference. 
# Having a copy makes it easier to later on see what is changed and what not.

# attacks_clean = attacks_drop_rows.copy(deep=True)

In [ ]:
# attacks_clean

In [ ]:
# Displaying information on the columns, their types and the count of non-null values.

# attacks_clean.info()



In [ ]:
# # Lowercase all strings and strip whitespace and/or quotationmarks around strings

# attacks_clean = attacks_clean.applymap(lambda x: x.lower() if isinstance(x, str) else x)
# attacks_clean = attacks_clean.applymap(lambda x: x.strip('" ') if type(x)==str else x)

In [ ]:
# Check column 'Case' for missing values

# find_non_default_missing_values(attacks_clean, 'Case', 'string')

# Check the non-unique values to see what is going on.

# attacks_clean[attacks_clean.duplicated('Case', keep=False)]

# They might indicatie cases where more then one person was attacked. 
# The columns regarding location and time might help here
# Since there are few and they seem irrelevant to our questions I will leave them for now

# DONE

In [ ]:
# Check column 'Type' for missing values

# find_non_default_missing_values(attacks_clean, 'Type', 'string')

# Changing NaN to None and the different spellings of boat accidents to boat.

# attacks_clean = attacks_clean.replace({'Type' : {np.nan : None, 'Boating' : 'Boat', 'Boatomg' : 'Boat'}})

# DONE

In [ ]:
# Check column 'Activity' for missing values

# find_non_default_missing_values(attacks_clean, 'Activity', 'string')

# Replacing with default values where needed
# attacks_clean['Activity'] = attacks_clean['Activity'].replace({'.': None})

# DONE


In [ ]:
# Check column 'Injury' for missing values

# find_non_default_missing_values(attacks_clean, 'Injury', 'string')

# DONE for now

In [ ]:
# Check column 'Species' for missing values

# find_non_default_missing_values(attacks_clean, 'Species', 'string')

In [ ]:
# Condensing all values to only the species

# All unique values in this column: 
# species_old = attacks_clean['Species'].unique()
# species_old

In [ ]:
# Thinking steps
# I need to fill a new column with: 
# if the word 'shark' occurs once: extract the word before 'shark' and display that plus 'shark'
# else: display 'needs looking at'


In [ ]:
# Return nurse shark
# string = 'said to involve a grey nurse shark that leapt out of the water and  seized the boy but species identification is questionable'
# word7 = "(\S+\s)?shark" # nurse shark

# species = re.search(word7, string)
# species.group()

In [ ]:
# Check column 'Fatal' for missing values

# find_non_default_missing_values(attacks_clean, 'Fatal', 'bool')

# display(attacks_clean.loc[(attacks_clean['Fatal']== False) | (attacks_clean['Fatal']== True)])

# Replacing values that are clear about their meaning with their boolean equivalents 

# attacks_clean['Fatal'] = attacks_clean['Fatal'].replace(dict.fromkeys(['y'], True))
# attacks_clean['Fatal'] = attacks_clean['Fatal'].replace(dict.fromkeys(['n'], False))
# replace_value(attacks_clean, 'Fatal', 'y', True)
# replace_value(attacks_clean, 'Fatal', 'n', False)

# Change values that are not clear to None for now, might remove them later.
# df[column_name].replace({missing_old: missing_new})

# non_bool = attacks_clean.loc[~((attacks_clean['Fatal'] == False) | (attacks_clean['Fatal'] == True))]
# attacks_clean['Fatal'] = attacks_clean['Fatal'].replace({missing_old: missing_new})
# attacks_clean['Fatal'] = attacks_clean.loc[~((attacks_clean['Fatal'] == False) | (attacks_clean['Fatal'] == True))] = None



In [ ]:
# Check column 'Age' for missing values

# find_non_default_missing_values(attacks_clean, 'Age', 'string')

# Strip the leading/trailing whitespace and quotation marks.

# attacks_clean['Age'] = attacks_clean['Age'].str.strip('"\' \()\\')

# Replace \xa0 and np.nan with None
# attacks_clean['Age'] = attacks_clean['Age'].replace(dict.fromkeys(['\xa0',np.nan], None))

# There are a lot of unclear ages. Since we want to know if there is a difference 
# between children and adults I think it is best to divide them into adult and child

# Transform the column so all clear numericals are in the right categories
# attacks_clean.transform({'Age': str.capitalize, "price": lambda price: round(price * 1.1)})

